In [1]:
import hail as hl

# hl.init()

Loading BokehJS ...

In [2]:
# hl.utils.get_1kg('data/1kg/')
# hl.utils.get_hgdp('data/hgdp/')

In [3]:
data_path = 'data/hgdp'

In [4]:
mt = hl.read_matrix_table(f'{data_path}/HGDP.mt')

Initializing Hail with default parameters...
Running on Apache Spark version 3.5.1
SparkUI available at http://c2843211b84f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.131-11d9b2ff89da
LOGGING: writing to /home/jovyan/hail-20240625-1647-0.2.131-11d9b2ff89da.log


In [5]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        QUALapprox: int32, 
        SB: array<int32>, 
        MQ: float64, 
        MQRankSum: float64, 
        VarDP: int32, 
        AS_ReadPosRankSum: float64, 
        AS_pab_max: float64, 
        AS_QD: float64, 
        AS_MQ: float64, 
        QD: float64, 
        AS_MQRankSum: float64, 
        FS: float64, 
        AS_FS: float64, 
        ReadPosRankSum: float64, 
        AS_QUALapprox: int32, 
        AS_SB_TABLE: array<int32>, 
        AS_VarDP: int32, 
        AS_SOR: float64, 
        SOR: float64, 
        transmitted_singleton: bool, 
        omni: bool, 
        mills: bool, 
        monoallelic: bool, 
        AS_VQSLOD: float64, 
        Inbreeding

In [6]:
populations_table = hl.import_table(f'{data_path}/HGDP_annotations.txt', key='s')
populations_table.show()

2024-06-25 16:48:02.361 Hail: INFO: Reading table without type imputation
  Loading field 's' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'continental_pop' as type str (not specified)
  Loading field 'sex_karyotype' as type str (not specified)
2024-06-25 16:48:21.857 Hail: WARN: aggregate_cols(): Aggregates over cols ordered by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2024-06-25 16:48:23.074 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'
2024-06-25 16:48:26.542 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-06-25 16:48:26.916 Hail: INFO: Coerced sorted dataset
2024-06-25 16:48:27.959 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-06-25 16:48:36.940 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-06-25 16:48:37.185 

,,,
s,pop,continental_pop,sex_karyotype
str,str,str,str
"""HG00096""","""gbr""","""nfe""","""XY"""
"""HG00097""","""gbr""","""nfe""","""XX"""
"""HG00099""","""gbr""","""nfe""","""XX"""
"""HG00100""","""gbr""","""nfe""","""XX"""
"""HG00101""","""gbr""","""nfe""","""XY"""
"""HG00102""","""gbr""","""nfe""","""XX"""
"""HG00103""","""gbr""","""nfe""","""XY"""
"""HG00105""","""gbr""","""nfe""","""XY"""


In [7]:
mt = mt.annotate_cols(**populations_table[mt.s])

In [8]:
hl.summarize_variants(mt)

Number of alleles,Count
2,10441
Allele type,Count
SNP,10441
Metric,Value
Transitions,6602
Transversions,3839
Ratio,1.72
Contig,Count
chr1,881
chr2,799


In [9]:
import random
import pandas as pd

rsid_list = mt.aggregate_rows(hl.agg.collect_as_set(mt.rsid))
selected_snps = random.choices(list(rsid_list), k=100)
mt_filtered = mt.filter_rows(hl.literal(selected_snps).contains(mt.rsid))

In [10]:
mt_filtered.count_rows()

99

In [11]:
def gt_freq_estimates(mt, group):
    p_ML = ((2 * hl.agg.sum(mt.GT.is_hom_ref())) + hl.agg.sum(mt.GT.is_het())) / (2 * hl.agg.count_where(hl.is_defined(mt.GT)))
    return mt.aggregate_entries(
        hl.agg.group_by(
            mt.rsid,
            hl.agg.group_by(
                group,
                hl.struct(
                    p_ML = p_ML,
                    p_AA = p_ML ** 2,
                    p_AB = 2 * p_ML * (1 - p_ML),
                    p_BB = (1 - p_ML) ** 2,
                )
            )
        )
    )

def create_freqs_dataframe(snp_freqs):
    rows = [
        {
            'rsid': rsid,
            'pop_freqs': hl.struct(**pop_freqs)
        }
        for rsid, pop_freqs in snp_freqs.items()
    ]
    df = pd.DataFrame(rows)
    return df

snp_freqs = gt_freq_estimates(mt_filtered, mt_filtered.pop)
freqs_ht = hl.Table.from_pandas(create_freqs_dataframe(snp_freqs), key='rsid')
freqs_ht.show()

+----------------+--------------------+--------------------+
| rsid           | pop_freqs.acb.p_ML | pop_freqs.acb.p_AA |
+----------------+--------------------+--------------------+
| str            |            float64 |            float64 |
+----------------+--------------------+--------------------+
| "rs1036894983" |           1.00e+00 |           1.00e+00 |
| "rs10422355"   |           5.50e-01 |           3.03e-01 |
| "rs1109855"    |           4.00e-01 |           1.60e-01 |
| "rs111930918"  |           1.00e+00 |           1.00e+00 |
| "rs112395787"  |           9.50e-01 |           9.03e-01 |
| "rs112488762"  |           9.50e-01 |           9.03e-01 |
| "rs112533403"  |           2.00e-01 |           4.00e-02 |
| "rs112730181"  |           8.50e-01 |           7.22e-01 |
| "rs112832077"  |           9.50e-01 |           9.03e-01 |
| "rs113145131"  |           1.00e+00 |           1.00e+00 |
+----------------+--------------------+--------------------+

+--------------------+--------------------+-----------------------+
| pop_freqs.acb.p_AB | pop_freqs.acb.p_BB | pop_freqs.adygei.p_ML |
+--------------------+--------------------+-----------------------+
|            float64 |            float64 |               float64 |
+--------------------+--------------------+-----------------------+
|           0.00e+00 |           0.00e+00 |              1.00e+00 |
|           4.95e-01 |           2.02e-01 |              2.50e-01 |
|           4.80e-01 |           3.60e-01 |              7.50e-01 |
|           0.00e+00 |           0.00e+00 |              1.00e+00 |
|           9.50e-02 |           2.50e-03 |              1.00e+00 |
|           9.50e-02 |           2.50e-03 |              1.00e+00 |
|           3.20e-01 |           6.40e-01 |              1.00e+00 |
|           2.55e-01 |           2.25e-02 |              1.00e+00 |
|           9.50e-02 |           2.50e-03 |              1.00e+00 |
|           0.00e+00 |           0.00e+00 |              1.00e+00 |
+--------------------+--------------------+-----------------------+

+-----------------------+-----------------------+-----------------------+
| pop_freqs.adygei.p_AA | pop_freqs.adygei.p_AB | pop_freqs.adygei.p_BB |
+-----------------------+-----------------------+-----------------------+
|               float64 |               float64 |               float64 |
+-----------------------+-----------------------+-----------------------+
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              6.25e-02 |              3.75e-01 |              5.63e-01 |
|              5.63e-01 |              3.75e-01 |              6.25e-02 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
|              1.00e+00 |              0.00e+00 |              0.00e+00 |
+-----------------------+-----------------------+-----------------------+

+--------------------+--------------------+--------------------+
| pop_freqs.asw.p_ML | pop_freqs.asw.p_AA | pop_freqs.asw.p_AB |
+--------------------+--------------------+--------------------+
|            float64 |            float64 |            float64 |
+--------------------+--------------------+--------------------+
|           1.00e+00 |           1.00e+00 |           0.00e+00 |
|           5.63e-01 |           3.16e-01 |           4.92e-01 |
|           6.88e-01 |           4.73e-01 |           4.30e-01 |
|           1.00e+00 |           1.00e+00 |           0.00e+00 |
|           8.75e-01 |           7.66e-01 |           2.19e-01 |
|           8.13e-01 |           6.60e-01 |           3.05e-01 |
|           2.50e-01 |           6

In [12]:
total_subjects = mt.count_cols()
ethnicity_counts = mt.aggregate_cols(hl.agg.counter(mt.pop))
ethnicity_proportions = {k: v / total_subjects for k, v in ethnicity_counts.items()}

ethnicity_proportions, sum(ethnicity_proportions.values())

({'acb': 0.025510204081632654,
  'adygei': 0.00510204081632653,
  'asw': 0.02040816326530612,
  'balochi': 0.002551020408163265,
  'basque': 0.00510204081632653,
  'beb': 0.02295918367346939,
  'bedouin': 0.017857142857142856,
  'brahui': 0.007653061224489796,
  'burusho': 0.01020408163265306,
  'cambodian': 0.007653061224489796,
  'cdx': 0.02806122448979592,
  'ceu': 0.04081632653061224,
  'chb': 0.012755102040816327,
  'chs': 0.02295918367346939,
  'clm': 0.02295918367346939,
  'colombian': 0.002551020408163265,
  'dai': 0.00510204081632653,
  'daur': 0.002551020408163265,
  'druze': 0.01020408163265306,
  'esn': 0.04336734693877551,
  'fin': 0.02295918367346939,
  'french': 0.00510204081632653,
  'gbr': 0.02295918367346939,
  'gih': 0.02806122448979592,
  'gwd': 0.04081632653061224,
  'han': 0.012755102040816327,
  'hazara': 0.007653061224489796,
  'hezhen': 0.002551020408163265,
  'ibs': 0.02806122448979592,
  'itu': 0.017857142857142856,
  'japanese': 0.01020408163265306,
  'jpt':

In [13]:
def annotate_rows_with_freqs(mt, freqs_ht):
    mt = mt.annotate_rows(
        freqs=freqs_ht[mt_filtered.rsid].pop_freqs
    )
    return mt

mt_annotated = annotate_rows_with_freqs(mt_filtered, freqs_ht)
mt_annotated.entries().show(3)

+---------------+------------+---------------+-----------+----------+
| locus         | alleles    | rsid          |      qual | filters  |
+---------------+------------+---------------+-----------+----------+
| locus<GRCh38> | array<str> | str           |   float64 | set<str> |
+---------------+------------+---------------+-----------+----------+
| chr1:13436332 | ["C","G"]  | "rs974190053" | -1.00e+01 | {}       |
| chr1:13436332 | ["C","G"]  | "rs974190053" | -1.00e+01 | {}       |
| chr1:13436332 | ["C","G"]  | "rs974190053" | -1.00e+01 | {}       |
+---------------+------------+---------------+-----------+----------+

+-----------------+---------------------------+----------+----------------+
| info.QUALapprox | info.SB                   |  info.MQ | info.MQRankSum |
+-----------------+---------------------------+----------+----------------+
|           int32 | array<int32>              |  float64 |        float64 |
+-----------------+---------------------------+----------+----------------+
|         2424012 | [42774,42087,45925,46682] | 6.00e+01 |      -3.10e-02 |
|         2424012 | [42774,42087,45925,46682] | 6.00e+01 |      -3.10e-02 |
|         2424012 | [42774,42087,45925,46682] | 6.00e+01 |      -3.10e-02 |
+-----------------+---------------------------+----------+----------------+

+------------+------------------------+-----------------+------------+
| info.VarDP | info.AS_ReadPosRankSum | info.AS_pab_max | info.AS_QD |
+------------+------------------------+-----------------+------------+
|      int32 |                float64 |         float64 |    float64 |
+------------+------------------------+-----------------+------------+
|     177468 |               3.06e-01 |        1.00e+00 |   1.37e+01 |
|     177468 |               3.06e-01 |        1.00e+00 |   1.37e+01 |
|     177468 |               3.06e-01 |        1.00e+00 |   1.37e+01 |
+------------+------------------------+-----------------+------------+

+------------+----------+-------------------+----------+------------+
| info.AS_MQ |  info.QD | info.AS_MQRankSum |  info.FS | info.AS_FS |
+------------+----------+-------------------+----------+------------+
|    float64 |  float64 |           float64 |  float64 |    float64 |
+------------+----------+-------------------+----------+------------+
|   6.00e+01 | 1.37e+01 |         -2.60e-02 | 0.00e+00 |   0.00e+00 |
|   6.00e+01 | 1.37e+01 |         -2.60e-02 | 0.00e+00 |   0.00e+00 |
|   6.00e+01 | 1.37e+01 |         -2.60e-02 | 0.00e+00 |   0.00e+00 |
+------------+----------+-------------------+----------+------------+

+---------------------+--------------------+---------------------------+
| info.ReadPosRankSum | info.AS_QUALapprox | info.AS_SB_TABLE          |
+---------------------+--------------------+---------------------------+
|             float64 |              int32 | array<int32>              |
+---------------------+--------------------+---------------------------+
|            2.84e-01 |            2424012 | [42774,42087,45925,46682] |
|            2.84e-01 |            2424012 | [42774,42087,45925,46682] |
|            2.84e-01 |            2424012 | [42774,42087,45925,46682] |
+---------------------+--------------------+---------------------------+

+---------------+-------------+----------+----------------------------+
| info.AS_VarDP | info.AS_SOR | info.SOR | info.transmitted_singleton |
+---------------+-------------+----------+----------------------------+
|         int32 |     float64 |  float64 |                       bool |
+---------------+-------------+----------+----------------------------+
|        177468 |    6.94e-01 | 6.94e-01 |                      False |
|        177468 |    6.94e-01 | 6.94e-01 |                      False |
|        177468 |    6.94e-01 | 6.94e-01 |                      False |
+---------------+-------------+----------+----------------------------+

+-----------+------------+------------------+----------------+
| info.omni | info.mills | info.monoallelic

In [14]:
def non_nan(x, defailt=0.0):
    return hl.if_else(hl.is_finite(x), x, defailt)

In [15]:
def log_likelihood(genotype, freqs, pop):
    return (
        hl.case()
        .when(genotype.is_hom_ref(), non_nan(hl.log(freqs[pop].p_AA)))
        .when(genotype.is_het(), non_nan(hl.log(freqs[pop].p_AB)))
        .when(genotype.is_hom_var(), non_nan(hl.log(freqs[pop].p_BB)))
        .default(0.0)
    )

def get_log_likelihoods(mt, proportions):
    mt = mt.annotate_cols(
        log_likelihoods=hl.struct(**{
            pop: hl.agg.sum(log_likelihood(mt.GT, mt.freqs, pop)) + hl.log(proportions[pop])
            for pop in proportions
        })
    )
    return mt


mt_likelihood = get_log_likelihoods(mt_annotated, ethnicity_proportions)

In [16]:
def log_sum_exp(log_values):
    max_log = hl.max(log_values)
    return max_log + hl.log(hl.sum([hl.exp(log_value - max_log) for log_value in log_values]))


def calculate_posteriors(mt, proportions):
    log_likelihoods = [mt.log_likelihoods[pop] for pop in proportions.keys()]

    mt = mt.annotate_cols(log_likelihood_sum=log_sum_exp(log_likelihoods))
    return mt.annotate_cols(
        posteriors=hl.dict({
            pop: mt.log_likelihoods[pop] - mt.log_likelihood_sum
               for pop in proportions.keys()
        })
    )

mt_probs = calculate_posteriors(mt_likelihood, ethnicity_proportions)
mt_probs.cols().show(4)

+-----------+-------+-----------------+---------------+---------------------+
| s         | pop   | continental_pop | sex_karyotype | log_likelihoods.acb |
+-----------+-------+-----------------+---------------+---------------------+
| str       | str   | str             | str           |             float64 |
+-----------+-------+-----------------+---------------+---------------------+
| "HG00107" | "gbr" | "nfe"           | "XY"          |           -4.10e+01 |
| "HG00114" | "gbr" | "nfe"           | "XY"          |           -5.72e+01 |
| "HG00121" | "gbr" | "nfe"           | "XX"          |           -6.22e+01 |
| "HG00127" | "gbr" | "nfe"           | "XX"          |           -5.14e+01 |
+-----------+-------+-----------------+---------------+---------------------+

+------------------------+---------------------+-------------------------+
| log_likelihoods.adygei | log_likelihoods.asw | log_likelihoods.balochi |
+------------------------+---------------------+-------------------------+
|                float64 |             float64 |                 float64 |
+------------------------+---------------------+-------------------------+
|              -4.15e+01 |           -3.79e+01 |               -1.57e+01 |
|              -3.94e+01 |           -5.16e+01 |               -1.98e+01 |
|              -4.40e+01 |           -5.30e+01 |               -2.05e+01 |
|              -4.35e+01 |           -4.63e+01 |               -2.12e+01 |
+------------------------+---------------------+-------------------------+

+------------------------+---------------------+-------------------------+
| log_likelihoods.basque | log_likelihoods.beb | log_likelihoods.bedouin |
+------------------------+---------------------+-------------------------+
|                float64 |             float64 |                 float64 |
+------------------------+---------------------+-------------------------+
|              -3.44e+01 |           -4.31e+01 |               -4.36e+01 |
|              -2.89e+01 |           -4.25e+01 |               -3.57e+01 |
|              -3.28e+01 |           -4.77e+01 |               -4.20e+01 |
|              -3.48e+01 |           -4.97e+01 |               -4.78e+01 |
+------------------------+---------------------+-------------------------+

+------------------------+-------------------------+---------------------------+
| log_likelihoods.brahui | log_likelihoods.burusho | log_likelihoods.cambodian |
+------------------------+-------------------------+---------------------------+
|                float64 |                 float64 |                   float64 |
+------------------------+-------------------------+---------------------------+
|              -3.67e+01 |               -4.08e+01 |                 -3.15e+01 |
|              -3.40e+01 |               -3.94e+01 |                 -3.63e+01 |
|              -3.72e+01 |               -4.17e+01 |                 -3.27e+01 |
|              -4.12e+01 |               -4.03e+01 |                 -3.43e+01 |
+------------------------+-------------------------+---------------------------+

+---------------------+---------------------+---------------------+
| log_likelihoods.cdx | log_likelihoods.ceu | log_likelihoods.chb |
+---------------------+---------------------+---------------------+
|             float64 |             float64 |             float64 |
+---------------------+---------------------+---------------------+
|           -4.73e+01 |           -3.99e+01 |           -3.65e+01 |
|           -4.97e+01 |           -3.85e+01 |           -3.09e+01 |
|           -4.99e+01 |           -4.29e+01 |           -3.58e+01 |
|           -5.01e+01 |           -4.47e+01 |           -3.90e+01 |
+---------------------+---------------------+---------------------+

+---------------------+---------------------+---------------------------+
| log_likelihoods.chs | log_likelihoods.clm | log_likelihoods.colombian |
+---------------------+---------------------+---------------------------+
|  

In [17]:
def predict_ancestry(posteriors):
    return hl.bind(
        lambda x: hl.sorted(x.items(), key=lambda item: item[1], reverse=True)[0][0],
        posteriors
    )

mt_probs = mt_probs.annotate_cols(predicted_ancestry=predict_ancestry(mt_probs.posteriors))
mt_probs.cols().select('predicted_ancestry', 'pop').show(5)

,,
s,predicted_ancestry,pop
str,str,str
"""HG00107""","""colombian""","""gbr"""
"""HG00114""","""colombian""","""gbr"""
"""HG00121""","""colombian""","""gbr"""
"""HG00127""","""colombian""","""gbr"""
"""HG00132""","""colombian""","""gbr"""


In [ ]:
mt_probs.aggregate_cols(predicted_ancestry=predict_ancestry(mt_probs.posteriors))

In [18]:
tp, total = mt_probs.filter_cols(mt_probs.predicted_ancestry == mt_probs.pop).count_cols(), mt_probs.count_cols()

tp / total, tp, total

(0.00510204081632653, 2, 392)

In [19]:
def calculate_error_rate(mt):
    correct_predictions = mt.filter_cols(mt.predicted_ancestry == mt.pop)
    total_samples = mt.count_cols()
    error_rate = correct_predictions.aggregate_cols(
        hl.agg.sum(1 - hl.exp(correct_predictions.posteriors[correct_predictions.predicted_ancestry]))
    ) / total_samples
    return error_rate


calculate_error_rate(mt_probs)

0.0013290751226235143